# Working with Verily Workbench groups

<table align="left">

  <td>
    <a href="https://github.com/verily-src/workbench-examples/blob/main/first_hour_on_vwb/working_with_groups.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>                                                                                             
</table>

## Overview

Access to resources and workspaces in Verily Workbench is handled by [Verily Workbench groups](https://support.workbench.verily.com/docs/guides/admin/user_groups/). This notebook provides a series of widgets that enable you to perform various tasks related to groups in Verily Workbench. All of the tasks accomplished by widgets in this notebook can also be accomplished via [Workbench CLI](https://support.workbench.verily.com/docs/guides/cli/cli_intro/#installing-and-running-the-workbench-cli) commands in your cloud environment's termnal. 

### Objective

Perform common workspace resource operations including:

- [View the Verily Workbench groups in which you're a member](#view_groups)
- [Create a new Verily Workbench group](#create_group)
- [Delete an existing Verily Workbench group](#delete_group)
- [Add and remove collaborators to and from your Verily Workbench group](#manage_membership)

#### How to run this notebook

Please run the Setup section before running any other sections in this worksapce. 

#### Costs

This notebook takes less than a minute to run, which will typically cost less than $0.01 of compute time on your cloud environment.

## Setup

Run the cell below to set up utilities for the widgets provided in this workspace.

In [1]:
from IPython.display import display, HTML
from typing import List
import csv
import widget_utils as wu
import vwb_org_group_utils as vogu
import ipywidgets as widgets
import subprocess

## Legacy Verily Workbench group memberships
<a id='view_groups'></a>

On April 14, 2025, Workbench rolled out a change to the implementation of Workbench groups that allows users to create groups which are linked to their organization. Prior to this change, all groups were global in scope -- not linked to any organization. We refer to these global groups which predate this change as "legacy Workbench groups". To view any legacy Workbench groups in which you have a role, run the cell below.

In [2]:
result = subprocess.run(['wb','group','list','--format=JSON'],capture_output=True,text=True)
print(result.stderr) if not result.stdout else display(HTML(wu.list_legacy_groups(result.stdout)))

3p-sensors-workspace-write,['ADMIN']
aou-drc-gcp-pod,['MEMBER']
aou-initial-credit-stable-pod,['MEMBER']
aou-stable-rt,"['ADMIN', 'MEMBER']"
cdm-adhd-owner,['MEMBER']
cdm-demo-1-ws-owners,['MEMBER']
cdm-demo-1-ws-writers,['MEMBER']
cdm-sjogren-owner,['MEMBER']
cdm-workspace-write,['ADMIN']
colgate-ohs-vwb-dd-owners,['MEMBER']
colgate-ohs-vwb-demo-ws-owners,['ADMIN']


### View members of legacy Verily Workbench groups

In order to view the members of a legacy Verily Workbench group, you must be an admin of that group. To investigate the membership of your group, run the cell below to create a widget. Provide the group name in the widget input field and click the button to view a list of the members of the desired group.

In [3]:
class ListMembersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.output = widgets.Output()
        self.list_button = wu.StyledButton('List users in group','Click to list users in this Verily Workbench group.','list').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.list_button,self.output],layout=wu.vbox_layout)
        self.list_button.on_click(self.list_members)

        
    def list_members(self,b):
        self.output.clear_output()
        with self.output:
            wb_command = ["wb","group","list-users",f"--name={self.input_group_name.value}","--format=JSON"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            if not result.stdout:
                self.output.append_display_data(result.stderr)
            else:
                self.output.append_display_data(HTML(wu.list_legacy_group_members(result.stdout)))

# Create widget                
list_members_widget = ListMembersWidget()
display(list_members_widget.vb)

## Organization-linked Workbench groups

Starting on April 14, 2025, it is no longer possible to create "legacy" Workbench groups. Users must provide an organization when creating groups, and org admins and members can create org-linked groups. To view the org-linked Workbench groups in which you have a role, run the cell below.

In [4]:
class ListOrgGroupsWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the org ID for a Verily Workbench org.')
        self.input_org_id = wu.TextInputWidget('<ORG_ID>',"Org ID:").get()
        self.output = widgets.Output()
        self.list_button = wu.StyledButton('List org-linked group','Click to list org-linked groups.','list').get()
        self.vb = widgets.VBox([self.label,self.input_org_id,self.list_button,self.output],layout=wu.vbox_layout)
        self.list_button.on_click(self.list_groups)

        
    def list_groups(self,b):
        self.output.clear_output()
        with self.output:
            wb_command = ['wb', 'org','group','list',f"--org={self.input_org_id.value}","--format=JSON"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            if not result.stdout:
                self.output.append_display_data(result.stderr)
            else:
                self.output.append_display_data(HTML(wu.list_org_groups(result.stdout)))

# Create widget                
list_org_groups_widget = ListOrgGroupsWidget()
display(list_org_groups_widget.vb)

## View members of org-linked Verily Workbench groups

In order to view the members of a org-linked Verily Workbench group, you must either be an admin of that group or an org admin. To investigate the membership of your group, run the cell below to create a widget. Provide the *group name* and the *org ID* in the widget input fields and click the button to view a list of the members of the desired group.

In [5]:
class ListMembersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>',"Org ID:").get()
        self.output = widgets.Output()
        self.list_button = wu.StyledButton('List users in group','Click to list users in this Verily Workbench group.','list').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.list_button,self.output],layout=wu.vbox_layout)
        self.list_button.on_click(self.list_members)

        
    def list_members(self,b):
        self.output.clear_output()
        with self.output:
            self.output.append_display_data(HTML(vogu.get_flat_roles_html(vogu.get_org_linked_group_roles(self.input_org_id.value, self.input_group_name.value))))

# Create widget                
list_members_widget = ListMembersWidget()
display(list_members_widget.vb)

## Create a new Verily Workbench group
<a id='create_group'></a>

Run the cell below to create a widget that creates an org-linked Workbench group. Then, populate the widget's input fields:

- `Group Name`: Should be unique and reflect the purpose of the group (for example, for a group of Verily researchers working on the 1000 Genomes dataset, `verily-1000-genomes-researchers` would be appropriate).
- `Org ID`: Your organization's ID.
- `Expiration`: The default and maximimum MEMBER role expiration in *days*. Zero means the MEMBER role does not expire.
- `Expiration Notification`: Notify group admins about member expirations.
- `Require Grant Reason`: Require a reason on grants to the group.

Click the button to create a new Verily Workbench group. 
You should see output resembling:

```
Workbench group created.
<GROUP_NAME>
  Organization           : <ORG_ID>
  Description            : <DESCRIPTION>
  Expiration days        : <EXPIRATION_DURATION>
  Expiration notify      : <TRUE or FALSE>
  Sync to Google Group   : <TRUE>
  Grant reason required  : <TRUE or FALSE>
  Email                  : <PREFIX>_<GROUP_NAME>@verily-bvdp.com
  Current user's roles   : ADMIN
```

In [6]:
class CreateGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name & org ID for your Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_expiration_duration = wu.BoundIntInputWidget(0,0,366,'Expiration Duration in Days:').get()
        self.enable_expiration_notification = wu.LongLabelCheckbox('Enable notifications of expiration to group admins.').get()
        self.require_grant_reason = wu.LongLabelCheckbox('Require admins to provide a reason when granting a role on this group.').get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Create group','Click to create a Verily Workbench group.','plus').get()
        self.required_fields = [
            self.label,self.input_group_name,self.input_org_id,self.input_expiration_duration,
            self.enable_expiration_notification, self.require_grant_reason, self.button,self.output
        ]
        self.vb = widgets.VBox(children=self.required_fields,layout=wu.vbox_layout)
        self.button.on_click(self.create_group)

    def create_group(self,b):
        with self.output:
            self.output.clear_output()
            commandList = ["wb","group","create",
                           f"--name={self.input_group_name.value}",
                           f"--org={self.input_org_id.value}",
                           f"--expiration={int(self.input_expiration_duration.value)}",
                           "--format=text"]

            if self.enable_expiration_notification.value:
                commandList.append(f"--expiration-notification")
            if self.require_grant_reason.value:
                commandList.append(f"--require-grant-reason")
            result = subprocess.run(commandList,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
create_group_widget = CreateGroupWidget()
display(create_group_widget.vb)

## Delete a Verily Workbench group

<a id='delete_group'></a>

In order to delete a Verily Workbench group, you must be an admin of that group. Run the cell below to create a widget, then populate the input fields and click the button to delete the desired group.

**NOTE:** Once a Verily Workbench org-linked group has been deleted, this operation cannot be reversed. Please take care not to delete any groups currently in use.

Your output should resemble:
```
<GROUP_NAME>
  Email: <GROUP_NAME>@verily.com
  # Members: <NUMBER_OF_MEMBERS>
  Current user's policies: ADMIN
Workbench group successfully deleted.
```

In [7]:
class DeleteGroupWidget(object):
    def __init__(self):
        self.label = widgets.Label(value='Please provide the name for a Verily Workbench group.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Delete group','Click to delete a Verily Workbench group.','trash').get()
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.button,self.output],layout=wu.vbox_layout)
        self.button.on_click(self.delete_group)

    def delete_group(self,b):
        with self.output:
            self.output.clear_output()
            wb_command = ["wb","group","delete",f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}","--quiet"]
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
delete_group_widget = DeleteGroupWidget()
display(delete_group_widget.vb)

## Manage org-linked Verily Workbench group membership
<a id='manage_membership'></a>

Managing group membership is often an ongoing task throught a project's lifecycle.
<br>The sub-sections below create widgets that empower you to manage group membership without having to use the Workbench CLI in the terminal.

**Note:** In order to manage the membership of an org-linked Verily Workbench group, you must have the 'ADMIN' role for that group.

### Add a user to a Verily Workbench group

1. Run the cell below to create a widget that adds a user to a Verily Workbench group.
1. In the widget, provide the group name & org ID for an existing Verily Workbench group and the email of the user you wish to add. 
1. Click the button to add the user to the group!

You should then see output resembling:
```
Successfully granted role
Member of group <GROUP_NAME>:
  Scope: organization - <ORG_USER_FACING_ID>
  User: <USER_EMAIL>
  Roles: <ROLE>
```

In [8]:
class GrantUserGroupRoleWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_user_email = wu.TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.role_drop_down = wu.DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Role:").get()
        self.input_reason = wu.TextInputWidget('<REASON>', "Reason:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Add user','Click to add a user to the group.','user-plus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_org_id,
             self.input_user_email,
             self.role_drop_down,
             self.input_reason,
             self.button,
             self.output
            ],layout=wu.vbox_layout)
        self.button.on_click(self.add_user)

    def add_user(self,b):
        with self.output:
            wb_command = ["wb", "group", "role", "grant", "user", f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}",f"--email={self.input_user_email.value}",f"--role={self.role_drop_down.value}"]
            if self.require_grant_reason.value:
                commandList.append(f"--reason={self.input_reason.value}")
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
grant_user_role_widget = GrantUserGroupRoleWidget()
display(grant_user_role_widget.vb)

### Remove a user from an org-linked Verily Workbench group

1. Run the cell below to create a widget.
1. In the widget, provide the group name for an existing Verily Workbench group and the email of the user you wish to remove. 
1. Click the button to remove the user from the group.

You should see output resembling:
```
Successfully revoked <ROLE> role in group <GROUP_NAME> from user <USER_EMAIL>.
```

In [9]:
class RevokeUserRoleWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_user_email = wu.TextInputWidget('<USER_EMAIL>',"User Email:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.role_drop_down = wu.DropdownInputWidget(['MEMBER','ADMIN'],'MEMBER',"Role:").get()
        self.input_reason = wu.TextInputWidget('<REASON>', "Reason:").get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Remove user','Click to remove a user from the group.','user-minus').get()
        self.vb = widgets.VBox(
            [self.label,
             self.input_group_name,
             self.input_org_id,
             self.input_user_email,
             self.role_drop_down,
             self.input_reason,
             self.button,
             self.output
            ],layout=wu.vbox_layout)
        self.button.on_click(self.remove_user)

    def remove_user(self,b):
        with self.output:
            wb_command = ["wb", "group", "role", "revoke", "user", f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}",f"--email={self.input_user_email.value}",f"--role={self.role_drop_down.value}"]
            if self.require_grant_reason.value:
                commandList.append(f"--reason={self.input_reason.value}")
            result = subprocess.run(wb_command,capture_output=True,text=True)
            print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
revoke_user_role_widget = RevokeUserRoleWidget()
display(revoke_user_role_widget.vb)

### Add a batch of users to a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `org_id`: The target organization's ID.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "WORKBENCH_USER_EMAIL" and "ROLE", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding org-linked group role (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to add a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User added to Workbench group.
<EMAIL>: <POLICY>
```

In [10]:
class BatchAddUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:",).get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_file = wu.TextInputWidget('<FILE>',"CSV File:",).get()
        self.input_reason = wu.TextInputWidget('<REASON>', "Reason:",).get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Add users','Click to add a batch of users to the group.','user-plus').get()
        self.button.on_click(self.batch_add_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.input_reason,self.input_file,self.button,self.output],layout=wu.vbox_layout)

    def batch_add_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    wb_command = ["wb", "group", "role", "grant","user",f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}",f"--email={user['WORKBENCH_USER_EMAIL']}",f"--policy={user['ROLE']}"]
                    if self.input_reason.value:
                        commandList.append(f"--reason={self.input_reason.value}")
                    result = subprocess.run(wb_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)



# Instantiate widget
batch_add_users_widget = BatchAddUsersWidget()
display(batch_add_users_widget.vb)

### Remove a batch of users from a Verily Workbench group

Run the cell below to create a widget. The widget takes the following inputs:
- `group name`: The name of an existing Verily Workbench group of which you are an admin.
- `org_id`: The target organization's ID.
- `csv`: a CSV file located in your cloud environment in the same directory as this notebook. A template CSV file is provided in this directory, [batch_template.csv]('batch_template.csv'), consisting of the headers "WORKBENCH_USER_EMAIL" and "ROLE", followed by one line per user. Each line contains a user's Verily Workbench email and the corresponding membership policy (either "MEMBER" or "ADMIN").

Once you've created a CSV with your users' information and populated the input fields, click the button to remove a batch of users from the Verily Workbench group.

For each user, you should see output resembling:
```
User (<USER_EMAIL>) removed from policy (<POLICY>) in group (<GROUP_NAME>).
```

In [11]:
class BatchRemoveUsersWidget(object):
    def __init__(self):
        self.label = widgets.Label(value = 'Please provide appropriate values for the fields below.')
        self.input_group_name = wu.TextInputWidget('<GROUP_NAME>',"Group Name:").get()
        self.input_org_id = wu.TextInputWidget('<ORG_ID>', "Organization ID:").get()
        self.input_file = wu.TextInputWidget('<FILE>',"CSV File:").get()
        self.input_reason = wu.TextInputWidget('<REASON>', "Reason:",).get()
        self.output = widgets.Output()
        self.button = wu.StyledButton('Remove users','Click to remove a batch of users to the group.','user-minus').get()
        self.button.on_click(self.batch_remove_users)
        self.vb = widgets.VBox([self.label,self.input_group_name,self.input_org_id,self.input_reason,self.input_file,self.button,self.output],layout=wu.vbox_layout)

    def batch_remove_users(self,b):
        with self.output:
            self.output.clear_output()
            with open(self.input_file.value) as csv_file:
                csv_reader = csv.DictReader(csv_file,delimiter=',')
                for user in csv_reader:
                    wb_command = ["wb", "group", "role","revoke","user",f"--name={self.input_group_name.value}",f"--org={self.input_org_id.value}",f"--email={user['WORKBENCH_USER_EMAIL']}",f"--policy={user['POLICY']}"]
                    if self.input_reason.value:
                        commandList.append(f"--reason={self.input_reason.value}")
                    result = subprocess.run(wb_command,capture_output=True,text=True)
                    print(result.stderr) if not result.stdout else print(result.stdout)

# Instantiate widget
batch_remove_users_widget = BatchRemoveUsersWidget()
display(batch_remove_users_widget.vb)

## Provenance

Generate information about this notebook environment and the packages installed.

In [12]:
!date

Tue May 27 06:33:59 PM UTC 2025


Conda and pip installed packages:

In [13]:
!conda env export

name: base
channels:
  - conda-forge
dependencies:
  - _libgcc_mutex=0.1=conda_forge
  - _openmp_mutex=4.5=2_gnu
  - _r-mutex=1.0.1=anacondar_1
  - aiohappyeyeballs=2.5.0=pyhd8ed1ab_0
  - aiohttp=3.11.13=py310h89163eb_0
  - aiosignal=1.3.2=pyhd8ed1ab_0
  - alsa-lib=1.2.13=hb9d3cd8_0
  - annotated-types=0.7.0=pyhd8ed1ab_1
  - anyio=4.8.0=pyhd8ed1ab_0
  - aom=3.9.1=hac33072_0
  - archspec=0.2.5=pyhd8ed1ab_0
  - argon2-cffi=23.1.0=pyhd8ed1ab_1
  - argon2-cffi-bindings=21.2.0=py310ha75aee5_5
  - arrow=1.3.0=pyhd8ed1ab_1
  - asttokens=3.0.0=pyhd8ed1ab_1
  - async-lru=2.0.4=pyhd8ed1ab_1
  - async-timeout=5.0.1=pyhd8ed1ab_1
  - attr=2.5.1=h166bdaf_1
  - attrs=25.1.0=pyh71513ae_0
  - aws-c-auth=0.7.31=he1a10d6_2
  - aws-c-cal=0.7.4=hae4d56a_2
  - aws-c-common=0.9.29=hb9d3cd8_0
  - aws-c-compression=0.2.19=h2bff981_2
  - aws-c-event-stream=0.4.3=h19b0707_4
  - aws-c-http=0.8.10=h14a7884_2
  - aws-c-io=0.14.19=hc9e6898_1
  - aws-c-mqtt=0.10.7=hb8d5873_2
  - aws-c-s3=0.6.7=h666547d_0
  - aws-c-sd

JupyterLab extensions:

In [14]:
!jupyter labextension list

[W 2025-05-27 18:34:20.097 LabApp] 'kernel_spec_manager_class' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
JupyterLab v4.3.4
/opt/conda/share/jupyter/labextensions
        nbdime-jupyterlab v3.0.2 enabled OK
        jupyterlab-jupytext v1.4.3 enabled OK (python, jupytext)
        jupyterlab_pygments v0.3.0 enabled OK (python, jupyterlab_pygments)
        @jupyter-widgets/jupyterlab-manager v5.0.13 enabled OK (python, jupyterlab_widgets)
        @jupyterlab/git v0.50.2 enabled OK (python, jupyterlab-git)



Number of cores:

In [15]:
!grep ^processor /proc/cpuinfo | wc -l

4


Memory:

In [16]:
!grep "^MemTotal:" /proc/meminfo

MemTotal:       26672900 kB


---
Copyright 2025 Verily Life Sciences LLC

Use of this source code is governed by a BSD-style   
license that can be found in the LICENSE file or at   
https://developers.google.com/open-source/licenses/bsd